### Restart and Run All Function to do the whole pipeline!

In [ ]:
arregleu dill eta picklen problemie!

In [1]:
from IPython.display import HTML, Javascript

def restart_run_all():
    display(HTML(
        '''
            <script>
                code_show = false;
                IPython.notebook.kernel.restart();
                setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 1000)
                
            </script>
        '''
    ))
#restart_run_all()

In [2]:
import ctypes
import gc

def free():
    gc.collect()
    torch.cuda.empty_cache()
    libc = ctypes.CDLL("libc.so.6")
    libc.malloc_trim(0)
    gc.collect()
    torch.cuda.empty_cache()
    libc = ctypes.CDLL("libc.so.6")
    libc.malloc_trim(0)

In order to be able to jump cells if a condition is met

In [3]:
def skip(line, cell=None):
    '''Skips execution of the current line/cell if line evaluates to True.'''
    if eval(line):
        return

    get_ipython().run_cell(cell) 

def load_ipython_extension(shell):
    '''Registers the skip magic when the extension loads.'''
    shell.register_magic_function(skip, 'line_cell')

def unload_ipython_extension(shell):
    '''Unregisters the skip magic when the extension unloads.'''
    del shell.magics_manager.magics['cell']['skip']
    
    
load_ipython_extension(get_ipython())

Get state of pipeline:

In [4]:
exp_name="Noisy_Dataset_Embedders"

try:
    f = open(f"state_{exp_name}_Embedders.txt", "r")
    current_state = int(f.read())
    f.close()
    
    current_state+=1
except:
    current_state = 0

print(current_state)
skip_it = current_state==6

3


### Input Image Dataset to train the embedders

In [5]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.decomposition
import sklearn.manifold
import umap
import dill
import pandas as pd
import torch
import json
import os

In [6]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader

class ImageDataset(Dataset):
    def __init__(self, GT_file_path, images_dir_path):
        self.df_GTs = pd.DataFrame.from_dict(json.load(open(GT_file_path)))
        self.images_dir_path = images_dir_path
        self.len_data = len(self.df_GTs)

    def __len__(self):
        return self.len_data

    def __getitem__(self, idx):
        img_path = f"{self.images_dir_path}/IM_{self.df_GTs.iloc[idx,0]}_phiCR_{self.df_GTs.iloc[idx,1]}.png"
        image = read_image(img_path) #[1, 2X+1, 2X+1] torch tensor
        label = torch.Tensor([float(self.df_GTs.iloc[idx, 1])]).type(torch.float32) #[1] torch tensor of float32
        return image, label

Choose Dataset and **training** hyperparameters for the Embedding Generation

In [7]:
# Noisy Train set!
GT_file_path_train_noisy = f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TRAIN/GROUND_TRUTHS.json"
images_dir_path_train_noisy =f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TRAIN"

# Non-Noisy Train set!
GT_file_path_train_non_noisy = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/IMAGE_LIBRARY/NON_NOISY/TEST/GROUND_TRUTHS.json"
images_dir_path_train_non_noisy = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/IMAGE_LIBRARY/NON_NOISY/TEST"

# Test set
GT_file_path_test_noisy = f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TEST/GROUND_TRUTHS.json"
images_dir_path_test_noisy =f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/OUTPUT/NOISY/TEST"

# Non-Noisy Train set!
GT_file_path_test_non_noisy = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/IMAGE_LIBRARY/NON_NOISY/TEST/GROUND_TRUTHS.json"
images_dir_path_test_non_noisy = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/IMAGE_LIBRARY/NON_NOISY/TEST"

use_noisy=True
num_images= 5000
num_decimals = 3
random_seed = 666
n_jobs=11

emb_dims=10

save_stuff_path = f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/Embedders/Embedders_and_KNN/{exp_name}/"
os.makedirs(save_stuff_path, exist_ok=True)

In [8]:
if use_noisy:
    training_data = ImageDataset(GT_file_path_train_noisy, images_dir_path_train_noisy)
    #test_data = ImageDataset(GT_file_path_test_noisy, images_dir_path_test_noisy)
else:
    training_data = ImageDataset(GT_file_path_train_non_noisy, images_dir_path_train_non_noisy)
    #test_data = ImageDataset(GT_file_path_test_non_noisy, images_dir_path_test_non_noisy)

Generate proper finite dataset for Embedders! X,y

In [9]:
np.random.seed(random_seed)
random_indices = np.random.choice(range(len(training_data)), num_images, replace=False)
#random_indices = np.random.choice(range(2850), num_images, replace=False)
X21 = training_data[0][0].shape[1]
X = np.zeros( (num_images, X21**2), dtype=np.float32)
y = np.zeros((num_images), dtype=np.float64)

for j,idx in enumerate(random_indices):
    im, lab = training_data[idx]
    X[j, :] = im[0].flatten()
    y[j] = lab   

y_categoric = (np.around(y+np.pi, num_decimals)*10**num_decimals).astype(int)

free()

Testing data for KNN tests

In [10]:
'''
random_indices_test = np.random.choice(range(len(test_data)), num_images_test, replace=False)
#random_indices = np.random.choice(range(2850), num_images, replace=False)
X_test = np.zeros( (num_images_test, X21**2), dtype=np.float32)
y_test= np.zeros((num_images_test), dtype=np.float64)

for j,idx in enumerate(random_indices_test):
    im, lab = test_data[idx]
    X_test[j, :] = im[0].flatten()
    y_test[j] = lab   
'''

'\nrandom_indices_test = np.random.choice(range(len(test_data)), num_images_test, replace=False)\n#random_indices = np.random.choice(range(2850), num_images, replace=False)\nX_test = np.zeros( (num_images_test, X21**2), dtype=np.float32)\ny_test= np.zeros((num_images_test), dtype=np.float64)\n\nfor j,idx in enumerate(random_indices_test):\n    im, lab = test_data[idx]\n    X_test[j, :] = im[0].flatten()\n    y_test[j] = lab   \n'

# (B) Class to Use Embedding Spaces for KNN

In [11]:
args_sk_KNN = {'n_neighbors':3, 'weights':'distance', 'algorithm':'auto', 'leaf_size':50, 'p':2,
               'metric':'minkowski', 'n_jobs':n_jobs}

class KNN_Regressor():
    def __init__(self, embedder_func, args_sk_KNN):
        # ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’
        # ‘uniform’, ‘distance’
        self.embedder_func = embedder_func
        self.KNN = sk.neighbors.KNeighborsRegressor(n_neighbors=args_sk_KNN['n_neighbors'],
                    weights=args_sk_KNN['weights'], algorithm=args_sk_KNN['algorithm'],
                    leaf_size=args_sk_KNN['leaf_size'], p=args_sk_KNN['p'], 
                    metric=args_sk_KNN['metric'], n_jobs=args_sk_KNN['n_jobs'])
        self.fitted = False
    
    def fit(self, X, y, already_embedded_X=False): # X [N_samples, dim_feats], y [N_samples] # y can be to regression floats!
        if not already_embedded_X:
            X = self.embedder_func(X) # [N_samples, dim_feats]
        self.KNN = self.KNN.fit(X, y)
        self.fitted = True
    
    def score(self, X, y, already_embedded_X=False):
        if not already_embedded_X:
            X = self.embedder_func(X)
        return self.KNN.score(X,y)
        
    def predict(self, X, already_embedded_X=False):
        if not already_embedded_X:
            X = self.embedder_func(X)
        return self.KNN.predict(X)
    
class Sklearn_embedder():
    def __init__(self, embedder, preprocess_fct):
        self.preprocess_fct = preprocess_fct
        self.embedder = embedder
    
    def __call__(self, X):
        return self.embedder.transform( self.preprocess_fct(X) )

# (C) Generate Embedding Space Transformers
## Choose Embedder Instance

In [12]:
if current_state==0:
    # PCA -> Existe la incremental PCA por si es massa grande el dataset!
    args = {'exp':'PCA','emb_dims':emb_dims, "whiten":True}
    embedder = sk.decomposition.PCA(n_components=args['emb_dims'], whiten=args['whiten'], random_state=random_seed)

elif current_state==1:
    # KPCA
    args = {'exp':'KPCA_rbf', 'emb_dims':emb_dims, 'kernel':'rbf', 'fit_inverse':True, 'max_iter':100}
    # kernels: linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’, ‘precomputed’
    embedder = sk.decomposition.KernelPCA(n_components=args['emb_dims'], kernel=args['kernel'], 
                    fit_inverse_transform=args['fit_inverse'], max_iter=args['max_iter'], 
                            random_state=random_seed, n_jobs=n_jobs)
elif current_state==2:
    # LLE 
    args = {'exp':'LLE_standard',"method":"standard", "n_neighbors": 200, "emb_dims": emb_dims, 'max_iter':100}
    # Methods: standard, hessian, ltsa, modified (modified_tol) 
    embedder = sk.manifold.LocallyLinearEmbedding(method=args['method'], n_neighbors=args['n_neighbors'],
                  n_components=args['emb_dims'], max_iter=args['max_iter'], random_state=random_seed, n_jobs=n_jobs)

elif current_state==3:
    # UMAP -> uses y continous
    args = {'exp':'UMAP', 'emb_dims':emb_dims, 'min_dist':0.1, 'n_neighbors':300, 'metric':'hamming', 'n_epochs':None,
           'target_metric':'l2'}
    # Metrics: euclidean, canberra, cosine, manhattan, braycurtis, mahalanobis, hamming
    embedder = umap.UMAP(n_components=args['emb_dims'], min_dist=args['min_dist'], n_epochs=args['n_epochs'],
                n_neighbors=args['n_neighbors'], metric=args['metric'], random_state=random_seed, n_jobs=n_jobs,
                        target_metric=args['target_metric']) 

elif current_state==4:
    # ISOMAP
    args = {'exp':'ISOMAP', 'n_neighbors':200, 'emb_dims':emb_dims, 'max_iter':100, 'neighbors_algorithm':'auto', 'metric':'minkowski' }
    embedder = sk.manifold.Isomap( n_neighbors=args['n_neighbors'],n_components=args['emb_dims'],
                        max_iter=args['max_iter'], neighbors_algorithm=args['neighbors_algorithm'], n_jobs=n_jobs,
                        metric=args['metric'], p=2)

elif current_state==5:
    # NCA -> uses y categorical
    args = {'exp':'NCA', 'emb_dims':emb_dims, 'init':'auto', 'max_iter':100 }
    # init ‘auto’, ‘pca’, ‘lda’, ‘identity’, ‘random’
    embedder = sk.neighbors.NeighborhoodComponentsAnalysis(n_components=args['emb_dims'], init=args['init'],
                                    max_iter=args['max_iter'], random_state=random_seed)

elif current_state==6:
    args = {'exp':'RAW'}
    embedder = lambda X : X
    

elif current_state!=7:
    raise ValueError
# Triplet
#args = {'exp':'TRIPLET_CNN', 'emb_dims':10}

### Apply pre-embedding pre-process function to the dataset

In [13]:
%%skip $skip_it

pre_process_name = "normalize_to_max_and_iX"

import torch
assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def compute_intensity_gravity_centers_torch( images):
    """
        Expects input image to be an array of dimensions [N_imgs, h, w].
        It will return an array of gravity centers [N_imgs, 2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to array indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = torch.sum(images, dim=1) # weights for x [N_images, raw_width]
    intensity_in_h = torch.sum(images, dim=2) # weights for y [N_images, raw_height]
    total_intensity = intensity_in_h.sum(dim=1) # [N_images]

    # Compute mass center for intensity
    # [N_images, 2] (h_center,w_center)
    return torch.nan_to_num( torch.stack(
        (torch.matmul(intensity_in_h.float(), torch.arange(images.shape[1], 
                                    dtype=torch.float32, device=device))/total_intensity,
         torch.matmul(intensity_in_w.float(), torch.arange(images.shape[2], 
                                    dtype=torch.float32, device=device))/total_intensity),
        dim=1
        ), nan=0.0, posinf=None, neginf=None)


def compute_raws_to_centered_iXs_torch( images, X, device):

    g_raw = compute_intensity_gravity_centers_torch(images) # [ N_images, 2]

    # crop the iamges with size (X+1+X)^2 leaving the gravity center in
    # the central pixel of the image. In case the image is not big enough for the cropping,
    # a 0 padding will be made.
    centered_images = torch.zeros( ( images.shape[0], 2*X+1, 2*X+1),  dtype = images.dtype, 
                                  device=device)

    # we round the gravity centers to the nearest pixel indices
    g_index_raw = torch.round(g_raw).int() #[ N_images, 2]

    # obtain the slicing indices around the center of gravity
    # TODO -> make all this with a single array operation by stacking the lower and upper in
    # a new axis!!
    # [ N_images, 2 (h,w)]
    unclipped_lower = g_index_raw-X
    unclipped_upper = g_index_raw+X+1

    # unclipped could get out of bounds for the indices, so we clip them
    lower_bound = torch.clip( unclipped_lower.float(), min=torch.Tensor([[0,0]]).to(device),
                             max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
    upper_bound = torch.clip( unclipped_upper.float(), min=torch.Tensor([[0,0]]).to(device),
                             max=torch.Tensor(list(images.shape[1:])).unsqueeze(0).to(device)).int()
    # we use the difference between the clipped and unclipped to get the necessary padding
    # such that the center of gravity is left still in the center of the image
    padding_lower = lower_bound-unclipped_lower
    padding_upper = upper_bound-unclipped_upper

    # crop the image
    for im in range(g_raw.shape[0]):
        centered_images[im, padding_lower[ im, 0]:padding_upper[ im, 0] or None,
                                    padding_lower[ im, 1]:padding_upper[ im, 1] or None] = \
                  images[im, lower_bound[ im, 0]:upper_bound[ im, 0],
                                      lower_bound[ im, 1]:upper_bound[ im, 1]]

    return centered_images

import gc
def normalize_to_max_and_iX_input_output_flatten(images, dtype=np.float64,
                    iX_dev='cpu', out_dev='cpu', X=302, batch_size=100): # images expected to be [N_images, h, w]
    out = np.zeros((images.shape[0], images.shape[-1]), dtype=np.float64)
    images= images.reshape(-1, X*2+1, X*2+1).astype(dtype)/np.expand_dims( np.amax(images, axis=(-2,-1) ), (-2,-1) )
    for j in range(0, images.shape[0], batch_size):
        out[j:(j+batch_size)] = compute_raws_to_centered_iXs_torch( torch.from_numpy(images[j:(j+batch_size)]).to(device), X, device).to('cpu').numpy().reshape(batch_size, -1)
        torch.cuda.empty_cache()
        gc.collect()
    return out


preprocess_fct = normalize_to_max_and_iX_input_output_flatten

cuda:0


In [14]:
%%skip $skip_it
# the training images do not need to be normalized and iX, only normalized to max (already iX)

#X = preprocess_fct(X) # apply preprocess funct
X = X/np.expand_dims(np.amax(X, axis=(-1) ), 1)
free()

## Run Embedding Training and KNN Training after it
Note that the embedders alone can be also used as metric for the simulation coordinate descent or Nedler Mead.

And note also that the embedder is still required for inference in KNN.

In [15]:
%%skip $skip_it

date = datetime.now().strftime("%d_%m_%Y_%Hh%Mm%s")
f_name_emb = f"{args['exp']}_EMBEDDER_n_images_{num_images}_emb_dims_{emb_dims}_pre_process_{pre_process_name}_n_decimals_{num_decimals}_seed_{random_seed}_date_{date}.sav"
f_name_knn = f"{args['exp']}_KNN_n_images_{num_images}_emb_dims_{emb_dims}_pre_process_{pre_process_name}_n_decimals_{num_decimals}_seed_{random_seed}_date_{date}.sav"

Train Embedder

In [16]:
%%skip $skip_it

if args['exp']=='NCA': # then y categorical
    X_embedded = embedder.fit_transform(X, y=y_categoric)
elif args['exp']=='RAW':
    X_embedded = embedder(X)
elif args['exp']=='UMAP':
    embedder = embedder.fit(X, y=y)
    X_embedded = embedder.embedding_
else: # in relaity only UMAP uses y from the rest, but it can handle y continous
    X_embedded = embedder.fit_transform(X, y=y)

del X
free()

/home/oiangu/anaconda3/lib/python3.8/site-packages/umap/umap_.py:1802: UserWarning: gradient function is not yet implemented for hamming distance metric; inverse_transform will be unavailable
  warn(


Save the embedder:

In [ ]:
with open(save_stuff_path+f_name_emb, 'wb') as f:
    dill.dump(embedder, f)

Train KNN

In [ ]:
%%skip $skip_it
free()

knn_regressor = KNN_Regressor( Sklearn_embedder( embedder, preprocess_fct), args_sk_KNN)
knn_regressor.fit(X_embedded, y, already_embedded_X=True)

Save trained embedder and knn

In [ ]:
%%skip $skip_it

with open(save_stuff_path+f_name_knn, 'wb') as f:
    dill.dump(knn_regressor, f)

Load Trained Embedder

In [ ]:
# embedder = dill.load((open(save_stuff_path+"NCA_EMBEDDER_n_images_500_emb_dims_10_pre_process_normalize_to_max_and_iX_n_decimals_3_seed_666_date_13_06_2022_18-M-28.sav", 'rb')))

Visualize resulting embedding in lower dimensions using PCA, first 2d PCA

In [ ]:
# X_embedded = embedder.transform(X)

In [ ]:
%%skip $skip_it

free()

from sklearn.decomposition import PCA
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(X_embedded)

Using Bokeh and px see 2d PCA

In [ ]:
%%skip $skip_it

import plotly.express as px
fig=px.scatter(np.concatenate((principalComponents, y[:,np.newaxis]), 1), x=0, y=1, color=2)

fig.update_layout(
    title=f"{args['exp']} -> PCA to 2D",
    xaxis_title="Principal component 1",
    yaxis_title="Principal component 2",
    coloraxis_colorbar=dict(title="phi CR",)
    )
fig.update_traces(marker={"size":5})
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.write_image(f"{save_stuff_path}/{f_name_emb}_2d.png")

fig.show()

free()

In [ ]:
'''
from io import BytesIO
from PIL import Image
import base64
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.palettes import Spectral10


output_notebook()

def embeddable_image(data):
    img_data = data.values.reshape(X21,X21).astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((64, 64), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

df['image'] = pd.DataFrame(data=X, columns=list(range(X.shape[1]))).apply(embeddable_image, axis=1)

datasource = ColumnDataSource(df)
color_mapping = LinearColorMapper(
    palette='Magma256',
    low=y.min(),
    high=y.max()
)

plot_figure = figure(
    title='PCA projection of the CR dataset',
    plot_width=800,
    plot_height=800,
    tools=('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>phiCR:</span>
        <span style='font-size: 18px'>@phiCR</span>
    </div>
</div>
"""))

plot_figure.circle(
    'principal component 1',
    'principal component 2',
    source=datasource,
    color=dict(field='phiCR', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=7
)
show(plot_figure)
'''

Now 3d PCA

In [ ]:
%%skip $skip_it
free()

from sklearn.decomposition import PCA
pca = PCA(n_components=3)

principalComponents = pca.fit_transform(X_embedded)

Using plotly

In [ ]:
%%skip $skip_it

fig=px.scatter_3d(np.concatenate((principalComponents, y[:,np.newaxis]), 1), x=0, y=1, z=2, color=3)
fig.update_layout(
    title=f"{args['exp']} -> PCA to 3D",
    scene = dict(
                    xaxis_title='princ. comp 1',
                    yaxis_title='princ. comp 2',
                    zaxis_title='princ. comp 3'),
    coloraxis_colorbar=dict(title="phi CR",)
    )

fig.update_traces(marker={"size":3})
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.write_image(f"{save_stuff_path}/{f_name_emb}_3d.png")

fig.show()

free()

Save just in case the generated embedding and the employed indices

In [ ]:
'''
# Initialize the hdf5 dataset saver
h5f = h5py.File(f"{save_stuff_path}/{args['exp_name']}_Training_Embedding_date_{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}.h5", 'a') # append if exists, create if not
h5f.create_dataset("Embeddings", data= embedded_training, compression="lzf", shuffle=True)
h5f.create_dataset("phiCRs", data= y, compression="lzf", shuffle=True)
h5f.create_dataset("ImageIndexes", data= random_indices, compression="lzf", shuffle=True)

h5f.flush()
'''

### If everything fine until here then updte state

In [ ]:
%%skip $skip_it

f = open(f"state_{exp_name}_Embedders.txt", "w")
f.write(str(current_state))
f.close()

## Restart kernel and re-run all

In [ ]:
%%skip $skip_it

restart_run_all()

### (D.2) Using Torch Embedder of Triplet Loss and train KNN

In [ ]:
import torch #should be installed by default in any colab notebook
import torch.nn as nn


assert torch.cuda.is_available(), "GPU is not enabled"

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


class Proximity_Metric_Based_On_Simple_Encoder(nn.Module):
    def __init__(self, X=302, feats_1=15, feats_2=20, feats_3=20, feats_4=20,
                 prop1=3, prop2=2, prop3=1, av_pool1_div=4, conv4_feat_size=15, av_pool2_div=10, 
                 out_fc_1=10, out_fc_2=10,
                 dropout_p1=0.2, dropout_p2=0.1
                ): 
        # propj is such that the_ image getting out from stage j is propj/prop_{j-1}-ths of the previous (with j=0 being 5)
        # clearly, prop_{j-1}>prop_{j}>...
        # 2X+1 will be assumed to be divisible by 5
        assert((2*X+1)%5==0)
        assert(prop1>prop2)
        assert(prop2>prop3)
        assert((int((prop3*(2*X+1)/5)/av_pool1_div)-conv4_feat_size)>0)
        
        
        super(Proximity_Metric_Based_On_Simple_Encoder, self).__init__()
        # in is [epoch_size, 1, 2X+1, 2X+1]
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=feats_1, 
                               kernel_size = int((2*X+1)/5*(5-prop1)+1), bias=True) 
        # out conv1 [epoch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
        self.conv2 = nn.Conv2d(in_channels=feats_1, out_channels=feats_2, 
                               kernel_size = int((2*X+1)/5*(prop1-prop2)+1), bias=True) 
        # out conv1 [epoch_size, feats_2, prop2*(prop1*(2X+1)/5)/prop1, prop2*(prop1*(2X+1)/5)/prop1]
        # that is [epoch_size, feats_2, prop2*(2X+1)/5), prop2*(2X+1)/5)]
        self.conv3 = nn.Conv2d(in_channels=feats_2, out_channels=feats_3, 
                               kernel_size = int((2*X+1)/5*(prop2-prop3)+1), bias=True)
        # out conv3 is [epoch_size, feats_3, prop3*(2X+1)/5), prop3*(2X+1)/5)]

        self.avPool1 = nn.AvgPool2d(kernel_size= int((prop3*(2*X+1)/5)*(1-1/av_pool1_div)) +1, stride=1)
        # out avpool1 is [epoch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]
        
        self.conv4 = nn.Conv2d(in_channels=feats_3, out_channels=feats_4, 
                              kernel_size= int((prop3*(2*X+1)/5)/av_pool1_div+1)-conv4_feat_size+1, bias=True)
        # [epoch_size, feats_4, conv4_feat_size, conv4_feat_size]
        
        self.avPool2 = nn.AvgPool2d(kernel_size= int(conv4_feat_size*(1-1/av_pool2_div)) +1, stride=1)
        # out avpool1 is [epoch_size, feats_4, conv4_feat_size/av_pool2_div+1, conv4_feat_size/av_pool2_div+1]
        
        #self.in_fc = int(feats_4*(conv4_feat_size/av_pool2_div+1)**2)
        self.in_fc = feats_4*((((((2*X+1-int((2*X+1)/5*(5-prop1)+1)+1)
                                  -int((2*X+1)/5*(prop1-prop2)+1)+1)
                                 -int((2*X+1)/5*(prop2-prop3)+1)+1)
                                -int((prop3*(2*X+1)/5)*(1-1/av_pool1_div)) -1+1)
                               -int((prop3*(2*X+1)/5)/av_pool1_div+1)+conv4_feat_size-1+1)
                              -int(conv4_feat_size*(1-1/av_pool2_div)) -1+1)**2
        
        self.fc1 = nn.Linear(in_features=self.in_fc, out_features=out_fc_1, bias=True)
        self.fc2 = nn.Linear(in_features=out_fc_1, out_features=out_fc_2, bias=True)
        
        self.dropout1 = nn.Dropout(p=dropout_p1, inplace=False)
        self.dropout2 = nn.Dropout(p=dropout_p2, inplace=False)
        self.relu = torch.nn.functional.leaky_relu

        self.batchNorm2 = nn.BatchNorm2d(num_features=feats_2)
        self.batchNorm4 = nn.BatchNorm2d(num_features=feats_4)

    def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
        x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]).float() # [batch_size, 1, 2X+1, 2X+1]
        # Normalize to unity the float image
        x = x/x.amax(dim=(2,3), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        
        x = self.relu( self.conv1(x) ) # [batch_size, feats_1, prop1*(2X+1)/5, prop1*(2X+1)/5]
        
        x = self.batchNorm2( self.relu( self.conv2(self.dropout1(x)) )) # [batch_size, feats_2, prop2*(2X+1)/5, prop2*(2X+1)/5]

        
        x = self.relu( self.conv3(self.dropout2(x)) ) # [batch_size, feats_3, prop3*(2X+1)/5, prop3*(2X+1)/5]

        
        x = self.avPool1(x) # [batch_size, feats_3, prop3*(2X+1)/5)/av_pool1_div, prop3*(2X+1)/5)/av_pool1_div]

        
        x = self.batchNorm4(self.conv4(self.dropout2(x))) # [batch_size, feats_4, conv4_feat_size, conv4_feat_size]

        
        x = self.relu( self.avPool2(x) ) # [batch_size, feats_4, conv4_feat_size/av_pool2_div, conv4_feat_size/av_pool2_div]

        
        x = x.view(x.shape[0], self.in_fc) #[batch_size, feats_4*int(conv4_feat_size/av_pool2_div)**2]

        
        x = self.fc2( self.relu( self.fc1(x) ) ) #[batch_size, out_fc_2]
        
        return x


class Proximity_Metric_Based_On_Corrector(nn.Module):
    def __init__(self, S0=2*302+1, S1=2*290+1, S2=2*250+1, S3=2*200+1, S4 = 2*10+1,
                 S5 = 2*1+1, S6 =2,
                 feats_S1=10, feats_S2=10, feats_S3=20, feats_S4=20, feats_S5 = 20,
                 out_fc1=100, out_fc2=10,
                 feats_S6 = 25,
                 dropout_p=0.1
                ): 
       
        super(Proximity_Metric_Based_On_Corrector, self).__init__()
        self.Ss = [S0, S1, S2, S3, S4, S5, S6]
        self.feats = [1, feats_S1, feats_S2, feats_S3, feats_S4, feats_S5, feats_S6]
        self.out_fc1 = out_fc1
        self.out_fc2 = out_fc2
        # in is [batch_size, 1, S0, S0]
        self.conv_S01 = nn.Conv2d(in_channels=1, out_channels=feats_S1, 
                               kernel_size = S0-S1+1, bias=True) 
        # out conv_S01 [batch_size, feats_S1, S1, S1]
        self.conv_S12 = nn.Conv2d(in_channels=feats_S1, out_channels=feats_S2, 
                               kernel_size = S1-S2+1, bias=True) 
        # out conv_S12 [batch_size, feats_S2, S2, S2]
        self.conv_S23 = nn.Conv2d(in_channels=feats_S2, out_channels=feats_S3, 
                               kernel_size = S2-S3+1, bias=True) 
        # out conv_S23 [batch_size, feats_S3, S3, S3]
        
        self.conv_S33 = nn.Conv2d(in_channels=feats_S3, out_channels=feats_S3, 
                               kernel_size = 1, bias=True) 
        # out conv_S33 [batch_size, feats_S3, S3, S3]
        
        self.conv_S34 = nn.Conv2d(in_channels=feats_S3, out_channels=feats_S4, 
                               kernel_size = S3-S4+1, bias=True) 
        # out conv_S34 [batch_size, feats_S4, S4, S4]
        
        self.conv_S45 = nn.Conv2d(in_channels=feats_S4, out_channels=feats_S5, 
                               kernel_size = S4-S5+1, bias=True) 
        # out conv_S45 [batch_size, feats_S5, S5, S5]
        self.conv_S56 = nn.Conv2d(in_channels=feats_S5, out_channels=feats_S6, 
                               kernel_size = S5-S6+1, bias=True) 
        # out conv_S56 [batch_size, feats_S6, S6, S6]
        
        self.in_fc1 = S6*S6*feats_S6
        self.fc1 = nn.Linear(in_features=self.in_fc1, out_features=out_fc1, bias=True)
        self.fc2 = nn.Linear(in_features=out_fc1, out_features=out_fc2, bias=True)
        
        self.dropout = nn.Dropout(p=dropout_p, inplace=False)
        self.relu = torch.nn.functional.leaky_relu

        self.batchNorm1 = nn.BatchNorm2d(num_features=feats_S3)
        self.batchNorm2 = nn.BatchNorm1d(num_features=out_fc1)
        

    def forward(self, x): # [batch_size, 2X+1, 2X+1] or [batch_size, 1, 2X+1, 2X+1]
        x = x.view(x.shape[0], 1, x.shape[-2], x.shape[-1]).float() # [batch_size, 1, 2X+1, 2X+1]
        # Normalize to unity the float image
        x = x/x.amax(dim=(2,3), keepdim=True)[0] # [batch_size, 1, 2X+1, 2X+1]
        
        # Conv layers
        x = self.relu(self.conv_S01(x)) # [batch_size, feats_S1, S1, S1]
        x = self.dropout( self.relu(self.conv_S12(x)) ) # [batch_size, feats_S2, S2, S2]
        x = self.relu(self.conv_S23(x)) # [batch_size, feats_S3, S3, S3]
        x = self.batchNorm1(self.relu(self.conv_S33(x)))
        x = self.relu(self.conv_S34(x))
        x = self.relu(self.conv_S45(x))
        x = self.relu(self.conv_S56(x))
        
        x = x.view(x.shape[0], self.in_fc1)
        x = self.dropout( self.relu(self.batchNorm2(self.fc1(self.dropout(x)))) )
        return self.fc2(x)

In [ ]:
args_NN_encoder = {'X':302, 'feats_1':20, 'feats_2':20, 'feats_3':20, 'feats_4':5, 'prop1':2.5, 'prop2':1.5,
                  'prop3':0.6, 'av_pool1_div':2, 'conv4_feat_size':8, 'av_pool2_div':10, 'out_fc_1':5,
                  'dropout_p1':0.2, 'dropout_p2':0.1, 'out_fc2':10} # out_fc_2 is the output dim of the embedding space
args_NN_denoiser = {'X':302, 'S0':2*302+1, 'S1':2*250+1, 'S2':2*200+1, 'S3':2*150+1, 'S4':2*10+1,
                    'S5':2*1+1, 'S6':2, 'feats_S1':5, 'feats_S2':5, 'feats_S3':10, 'feats_S4':20,
                    'feats_S5':20, 'feats_S6':25, 'out_fc1':100, 'dropout_p':0.1, 'out_fc_2':10}

saved_NN_path=f"/home/oiangu/Hippocampus/Conical_Refraction_Polarimeter/OUTPUT/LIBRARIES_OF_THEORETICAL_D/Basler_like_R0_300x_w0_300x_Z_50x_64bit/SIMULATIONS/Proximity_Metric/"

check_file="BEEEST1_BEST_Model_and_Optimizer_2022-05-03 17:24:03.205978_Proximity_Metric_from_Simple_Encoder_Batch_Hard_Soft_Margin.pt"
checkpoint = torch.load(saved_NN_path+f"/NNs/{check_file}")

class Triplet_NN_embedder(): # INPUT DATA IS ASSUMED TO BE NUMPY
    def __init__(self, args_NN_embedder, checkpoint_path, device, batch_size=200,
                 encoder_or_denoiser_based="encoder", output_to="numpy"):
        self.device = device
        self.output_to = output_to
        self.args_NN_embedder = args_NN_embedder
        self.checkpoint_path = checkpoint_path
        self.iX = args_NN_embedder['X']
        self.out_dim = args_NN_embedder['out_fc2']
        if encoder_or_denoiser_based=="encoder":
            self.model = Proximity_Metric_Based_On_Simple_Encoder( X=args_NN_embedder['X'], 
                feats_1=args_NN_embedder['feats_1'], feats_2=args_NN_embedder['feats_2'], 
                feats_3=args_NN_embedder['feats_3'], feats_4=args_NN_embedder['feats_4'],
                 prop1=args_NN_embedder['prop1'], prop2=args_NN_embedder['prop2'], prop3=args_NN_embedder['prop3'], 
                av_pool1_div=args_NN_embedder['av_pool1_div'], conv4_feat_size=args_NN_embedder['conv4_feat_size'], 
                av_pool2_div=args_NN_embedder['av_pool2_div'], 
                 out_fc_1=args_NN_embedder['out_fc_1'], out_fc_2=args_NN_embedder['out_fc2'],
                 dropout_p1=args_NN_embedder['dropout_p1'], dropout_p2=args_NN_embedder['dropout_p2'] )
        else:
            self.model = Proximity_Metric_Based_On_Corrector(S0=S0, S1=S1, S2=S2, S3=S3, S4=S4, S5=S5, S6=S6,
                 feats_S1=feats_S1, feats_S2=feats_S2, feats_S3=feats_S3, feats_S4=feats_S4,
                 feats_S5=feats_S5, feats_S6=feats_S6,
                 out_fc1=out_fc1, out_fc2=out_fc2,
                 dropout_p=dropout_p ) 
        
        self.preprocess = lambda X: (torch.tensor(X.reshape(X.shape[0],self.iX*2+1,self.iX*2+1)).to(device)) if len(X.shape)<3 else (torch.tensor(X).to(device))
        self.batch_size = batch_size

        # move model to gpu if available
        self.model.to(device)
        self.model.load_state_dict(checkpoint['model'])
        self.model.eval()
    
    @torch.no_grad() 
    def __call__(self, X):
        self.model.eval()
        if self.output_to=="numpy":
            Xout = np.zeros((X.shape[0], self.out_dim), dtype=np.float64)
            for j in range(0, X.shape[0], self.batch_size):
                Xout[j:(j+self.batch_size)] = self.model(self.preprocess(X[j:(j+self.batch_size)])).detach().to('cpu').numpy()
            return Xout
        else:
            return self.model(self.preprocess(X))      

### Train KNN for CNN embedder!

In [ ]:
triplet_embedder = Triplet_NN_embedder( args_NN_encoder, 
                checkpoint_path=save_stuff_path+check_file, 
                device=device, encoder_or_denoiser_based="encoder", output_to="numpy")

In [ ]:
knn_regressor = KNN_Regressor( -1, args_sk_KNN) # se introduzca el embedder en el código final, porke sino pesaría gigas este objeto en memoria

In [ ]:
X_emb = triplet_embedder(X)

In [ ]:
free()
knn_regressor.fit(  X_emb, y, already_embedded_X=True)
free()

In [ ]:
#X_test_emb = triplet_embedder(X_test)

In [ ]:
#knn_regressor.score( X_test_emb, y_test, already_embedded_X=True )

In [ ]:
#y_pred = knn_regressor.predict(X_test_emb, already_embedded_X=True)

In [ ]:
#np.mean(np.abs(y_pred-y_test))

In [ ]:
date = datetime.now().strftime("%d_%m_%Y_%Hh%Mm%Ss")
f_name_knn = f'Triplet_CNN_KNN_n_images_{num_images}_emb_dims_{10}_seed_{random_seed}_date_{date}.sav'
with open(save_stuff_path+f_name_knn, 'wb') as f:
    dill.dump(knn_regressor, f)

## Lets see generated embedding space after PCA

In [ ]:
from sklearn.decomposition import PCA
free()
# for plotting purposes
df = pd.DataFrame({'y':y})
df['phiCR'] = df["y"].astype(str)


pca = PCA(n_components=2)

principalComponents = pca.fit_transform(X_emb)

Using Bokeh and px see 2d PCA

In [ ]:
import plotly.express as px
fig=px.scatter(np.concatenate((principalComponents, y[:,np.newaxis]), 1), x=0, y=1, color=2)

fig.update_layout(
    title=f"{args['exp']} -> PCA to 2D",
    xaxis_title="Principal component 1",
    yaxis_title="Principal component 2",
    coloraxis_colorbar=dict(title="phi CR",)
    )
fig.update_traces(marker={"size":5})
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.write_image(f"{save_stuff_path}/{f_name_emb}_2d.png")

fig.show()
free()

In [ ]:
'''
from io import BytesIO
from PIL import Image
import base64
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.palettes import Spectral10
import plotly.express as px

output_notebook()

def embeddable_image(data):
    img_data = data.values.reshape(X21,X21).astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((64, 64), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

df['image'] = pd.DataFrame(data=X, columns=list(range(X.shape[1]))).apply(embeddable_image, axis=1)

datasource = ColumnDataSource(df)
color_mapping = LinearColorMapper(
    palette='Magma256',
    low=y.min(),
    high=y.max()
)

plot_figure = figure(
    title='PCA projection of the CR dataset',
    plot_width=800,
    plot_height=800,
    tools=('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>phiCR:</span>
        <span style='font-size: 18px'>@phiCR</span>
    </div>
</div>
"""))

plot_figure.circle(
    'principal component 1',
    'principal component 2',
    source=datasource,
    color=dict(field='phiCR', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=7
)
show(plot_figure)
'''

Now 3d PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
free()
principalComponents = pca.fit_transform(X_emb)

Using plotly

In [ ]:
fig=px.scatter_3d(np.concatenate((principalComponents, y[:,np.newaxis]), 1), x=0, y=1, z=2, color=3)
fig.update_layout(
    title=f"{args['exp']} -> PCA to 3D",
    scene = dict(
                    xaxis_title='princ. comp 1',
                    yaxis_title='princ. comp 2',
                    zaxis_title='princ. comp 3'),
    coloraxis_colorbar=dict(title="phi CR",)
    )

fig.update_traces(marker={"size":3})
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.write_image(f"{save_stuff_path}/{f_name_knn}_NN.png")

fig.show()
free()